In [22]:
import os, sys
import dotenv

dotenv.load_dotenv('../.env')    
PATH_DATA = os.environ.get('PATH_OPENSKY') + os.environ.get('PATH_REL_DATA')
PATH_USER_POSTS = PATH_DATA + 'user_posts/'

PATH_DATA


'/home/ties/Documents/00_Uni/24W/Foundations/opensky/data/'

In [23]:
import pandas as pd
import gzip 

def get_first_n_lines(file, n) -> pd.DataFrame:
    lines = []
    with gzip.open(file, 'r') as f:
        for i in range(n):
            line = f.readline().decode('utf-8').rstrip().split(',')
            lines.append(line)
    return pd.DataFrame(lines, columns=['user_id', 'replied_author', 'thread_root_author', 'reposted_author', 'quoted_author', 'date'])

df = get_first_n_lines(PATH_DATA + 'interactions.csv', 100000)
df

,user_id,replied_author,thread_root_author,reposted_author,quoted_author,date
0,836672,None,None,833271,None,202309192352
1,836672,None,None,61971,None,202310021913
2,836672,None,None,47191,None,202309231547
3,836672,None,None,17234,None,202309301358
4,836672,None,None,20490,None,202307261536
...,...,...,...,...,...,...
99995,375429,659696,375429,None,None,202309190508
99996,375429,235749,235749,None,None,202309181155
99997,375429,None,None,724986,None,202309171726
99998,375429,375429,375429,None,None,202309170808


In [24]:
#df = pd.read_csv(PATH_DATA + 'interactions.csv', compression='gzip')

In [25]:
df.replace('None', None, inplace=True)
df.isna().sum()

user_id                   0
replied_author        55482
thread_root_author    55482
reposted_author       45666
quoted_author         89381
date                      0
dtype: int64

In [26]:
# Values that are not None, per column
(-df.loc[:, ['replied_author', 'thread_root_author', 'reposted_author', 'quoted_author']].isna()).sum(axis=1).value_counts()

1    49480
2    47109
3     3353
4       58
Name: count, dtype: int64

In [27]:
df.user_id.value_counts().sort_values(ascending=False).head(10)

user_id
156317    18542
53893      5380
189823     3338
99362      3282
452381     2925
929828     2829
274263     2066
455474     1896
41057      1534
597989     1294
Name: count, dtype: int64

In [28]:
most_active_users = df.user_id.value_counts().head(10).index
mau = most_active_users[2]
mau

'189823'

In [29]:
PATH_DATA + 'user_posts/' + mau + '.jsonl'
#pd.read_json(PATH_DATA + '/user_posts/' + mau + '.jsonl')

'/home/ties/Documents/00_Uni/24W/Foundations/opensky/data/user_posts/189823.jsonl'

In [30]:
df.loc[:, "has_posts"] = df.user_id.apply(lambda x: os.path.isfile(PATH_USER_POSTS + x + '.jsonl'))
df

,user_id,replied_author,thread_root_author,reposted_author,quoted_author,date,has_posts
0,836672,None,None,833271,None,202309192352,False
1,836672,None,None,61971,None,202310021913,False
2,836672,None,None,47191,None,202309231547,False
3,836672,None,None,17234,None,202309301358,False
4,836672,None,None,20490,None,202307261536,False
...,...,...,...,...,...,...,...
99995,375429,659696,375429,None,None,202309190508,False
99996,375429,235749,235749,None,None,202309181155,False
99997,375429,None,None,724986,None,202309171726,False
99998,375429,375429,375429,None,None,202309170808,False


In [31]:
df.has_posts.value_counts()

has_posts
False    79476
True     20524
Name: count, dtype: int64

In [32]:
df.loc[df.has_posts].user_id.value_counts().sort_values(ascending=False).head(10)

user_id
53893     5380
3859      1267
531358     877
103559     800
291874     696
23812      634
660115     483
900081     376
434174     339
284652     270
Name: count, dtype: int64

In [33]:
most_active_users = df.loc[df.has_posts].user_id.value_counts().head(10)
mau = str(most_active_users[0])
mau

/tmp/ipykernel_34338/3127981004.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mau = str(most_active_users[0])


'5380'

In [34]:
df_posts = pd.read_json(PATH_DATA + '/user_posts/' + mau + '.jsonl', lines=True)
df_posts

,post_id,user_id,instance,date,text,langs,like_count,reply_count,repost_count,reply_to,replied_author,thread_root,thread_root_author,repost_from,reposted_author,quotes,quoted_author,labels,sent_label,sent_score
0,130696003,5380,bsky.social,1976-05-31 15:06:21.622,We’re vacationing in Mexico and went on a tour...,[eng],0,1,2,NaN,NaN,NaN,NaN,75802641.0,5379.0,NaN,NaN,NaN,2,0.805
1,37264927,5380,bsky.social,1976-05-31 15:06:21.728,Lots of vendors hawking souvenirs in Teotihuac...,[eng],5,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.903
2,130696004,5380,bsky.social,1976-05-31 15:06:10.142,Everything in Mexico is so colorful — even the...,[eng],8,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.971
3,130696005,5380,bsky.social,1976-05-31 15:06:00.001,Being adventurous in Mexico City! Trying ants...,[eng],2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.605
4,130696006,5380,bsky.social,1976-05-31 15:05:41.824,Saw this poster in Mexico City last night,[eng],3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,130696051,5380,bsky.social,1976-05-30 12:40:12.311,Chatting about a few new board games with Chri...,[eng],1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,75802734.0,5379.0,NaN,1,0.491
59,130696052,5380,bsky.social,1976-05-30 12:28:12.136,"Took Christina to Netflix Bites, a pop-up rest...",[eng],0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.621
60,130696053,5380,bsky.social,1976-05-30 12:27:41.848,Hooooly crap. RIP Florida education. https://w...,[eng],0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.952
61,130696054,5380,bsky.social,1976-05-30 12:27:32.122,Here's how we rolled last night in Call of Dut...,[eng],1,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0.850


In [35]:
df_posts.reply_to.value_counts().sort_values(ascending=False).head(10)

reply_to
3833758.0      1
130696016.0    1
70699216.0     1
130696037.0    1
63088355.0     1
130696050.0    1
Name: count, dtype: int64

In [36]:
def get_replies_to_users(df_posts : pd.DataFrame, user_ids : list):
    replied_to = df_posts.reply_to.unique()

    if len(replied_to) == 0:
        return None 
    
    overlap = list(set(replied_to).intersection(user_ids))
    return overlap

In [37]:
PATH_FEED_POSTS = PATH_DATA + 'feed_posts/'
df_news_feed = pd.read_json(PATH_FEED_POSTS + 'News.jsonl.gz', lines=True)
df_news_feed.drop_duplicates(subset=['post_id'], inplace=True)
df_news_feed

,post_id,user_id,instance,date,text,langs,like_count,reply_count,repost_count,reply_to,replied_author,thread_root,thread_root_author,quotes,quoted_author,labels
0,3517724,27155,com,1976-05-30 13:16:41.125,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,NaN,NaN,NaN,NaN,NaN,None
3,73976085,12030,com,1976-05-31 15:05:40.630,The low end consumer is still struggling\n\nht...,None,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,None
6,73975943,12030,com,1976-05-31 15:06:21.832,Signa creditors agree deep haircuts in wind do...,None,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,None
7,3753243,379,com,1976-05-31 15:06:22.341,"A conservative social media influencer, Isabel...",[eng],153,12,26,NaN,NaN,NaN,NaN,NaN,NaN,None
8,12776392,15717,ua,1976-05-31 15:06:22.336,💯 Income from frozen Russian assets would be b...,[eng],22,0,4,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42105,92231771,319,com,1976-05-30 12:11:11.434,The Constitution’s Elections Clause “does not ...,None,18,0,4,8486523.0,319.0,8486523.0,319.0,NaN,NaN,None
42107,70869659,319,com,1976-05-30 12:11:11.300,Tuesday briefing:\n\n- Vladimir Putin spoke fo...,None,18,0,3,NaN,NaN,NaN,NaN,NaN,NaN,None
42108,2974,319,com,1976-05-30 12:11:01.529,"no for real, no paywall on our articles until ...",None,78,0,50,1233314.0,319.0,1233314.0,319.0,NaN,NaN,None
42109,44325423,319,com,1976-05-30 12:10:11.843,Florida Gov. Ron DeSantis’s wealthy donors and...,None,19,6,6,NaN,NaN,NaN,NaN,NaN,NaN,None


In [21]:
news_feed_users = df_news_feed.user_id.unique()
get_replies_to_users(df_posts, news_feed_users)

set()

In [123]:
import numpy as np

np.array([np.array(lang) for lang in df_posts.langs.value_counts().index]).flatten()

array(['eng'], dtype='<U3')

In [125]:
df_user_langs = pd.DataFrame([int(user_file.split('.')[0]) for user_file in os.listdir(PATH_USER_POSTS)], columns = ['user_id'])
df_user_langs = df_user_langs.sample(1000)
df_user_langs

,user_id
91211,3319037
358015,495673
318434,2903128
523392,596854
191550,105709
...,...
585187,1321424
688498,1608959
600672,12130
364215,1062440


In [128]:
from tqdm import tqdm

tqdm.pandas()

def get_metadata(user_id : int) -> tuple:
    def get_df_posts(user_id: int) -> pd.DataFrame:
        global ERROR
        try:
            with open(PATH_USER_POSTS + str(user_id) + '.jsonl') as f:
                return pd.read_json(f, lines=True)
        except Exception as e:
            ERROR = True
            print(f"Exception in get_df_posts: {e}")
            return pd.DataFrame()

    def get_langs(df_posts: pd.DataFrame) -> str:
        global ERROR
        try:
            if df_posts.langs.dropna().empty:
                return None 
            return ",".join(list(np.unique(df_posts.langs.dropna().sum()).astype(str)))
        except Exception as e:
            ERROR = True
            print(f"Exception in get_langs: {e}")
            return None 

    def get_minmax_dates(df_posts: pd.DataFrame) -> tuple:
        global ERROR
        try:
            return df_posts.date.min(), df_posts.date.max()
        except Exception as e:
            ERROR = True
            print(f"Exception in get_minmax_dates: {e}")
            return None, None

    def get_mean_sent_score(df_posts: pd.DataFrame) -> float:
        global ERROR
        try:
            return df_posts.sent_score.mean()
        except Exception as e:
            ERROR = True
            print(f"Exception in get_mean_sent_score: {e}")
            return None

    def get_mean_like_count(df_posts: pd.DataFrame) -> float:
        global ERROR
        try:
            return df_posts.like_count.mean()
        except Exception as e:
            ERROR = True
            print(f"Exception in get_mean_like_count: {e}")
            return None

    def get_mean_reply_count(df_posts: pd.DataFrame) -> float:
        global ERROR
        try:
            return df_posts.reply_count.mean()
        except Exception as e:
            ERROR = True
            print(f"Exception in get_mean_reply_count: {e}")
            return None

    def get_mean_repost_count(df_posts: pd.DataFrame) -> float:
        global ERROR
        try:
            return df_posts.repost_count.mean()
        except Exception as e:
            ERROR = True
            print(f"Exception in get_mean_repost_count: {e}")
            return None
    
    ERROR = False
    req_cols = ['langs','nr_posts', 'date_min', 'date_max', 'mean_senti_score']

    try:
        with open(PATH_USER_POSTS + str(user_id) + '.jsonl') as f:
            df_posts = pd.read_json(f, lines=True)
    except Exception as e:
        print(e)
        ERROR = True
        ret_arr = [None, None, None, None, None, None, None, None, ERROR]
        assert len(ret_arr) == 9
        #return None, None, None, None, None, None, None, ERROR 

    return get_langs(df_posts), len(df_posts), *get_minmax_dates(df_posts), get_mean_sent_score(df_posts), get_mean_like_count(df_posts), get_mean_reply_count(df_posts), get_mean_repost_count(df_posts), ERROR

df_user_langs[['langs', 'nr_posts', 'date_min', 'date_max', 'mean_senti_score', 'mean_like_count', 'mean_reply_count', 'mean_repost_count', 'caught_exception']] = df_user_langs.user_id.progress_apply(get_metadata)

  0%|          | 0/1000 [00:00<?, ?it/s]

  6%|▌         | 62/1000 [00:00<00:13, 69.27it/s]

Exception in get_langs: 'DataFrame' object has no attribute 'langs'
Exception in get_minmax_dates: 'DataFrame' object has no attribute 'date'
Exception in get_mean_sent_score: 'DataFrame' object has no attribute 'sent_score'
Exception in get_mean_like_count: 'DataFrame' object has no attribute 'like_count'
Exception in get_mean_reply_count: 'DataFrame' object has no attribute 'reply_count'
Exception in get_mean_repost_count: 'DataFrame' object has no attribute 'repost_count'


 20%|█▉        | 196/1000 [00:03<00:12, 63.10it/s]

Exception in get_langs: 'DataFrame' object has no attribute 'langs'
Exception in get_minmax_dates: 'DataFrame' object has no attribute 'date'
Exception in get_mean_sent_score: 'DataFrame' object has no attribute 'sent_score'
Exception in get_mean_like_count: 'DataFrame' object has no attribute 'like_count'
Exception in get_mean_reply_count: 'DataFrame' object has no attribute 'reply_count'
Exception in get_mean_repost_count: 'DataFrame' object has no attribute 'repost_count'


...

Exception in get_langs: 'DataFrame' object has no attribute 'langs'
Exception in get_minmax_dates: 'DataFrame' object has no attribute 'date'
Exception in get_mean_sent_score: 'DataFrame' object has no attribute 'sent_score'
Exception in get_mean_like_count: 'DataFrame' object has no attribute 'like_count'
Exception in get_mean_reply_count: 'DataFrame' object has no attribute 'reply_count'
Exception in get_mean_repost_count: 'DataFrame' object has no attribute 'repost_count'


 99%|█████████▉| 990/1000 [00:16<00:00, 63.06it/s]

Exception in get_langs: 'DataFrame' object has no attribute 'langs'
Exception in get_minmax_dates: 'DataFrame' object has no attribute 'date'
Exception in get_mean_sent_score: 'DataFrame' object has no attribute 'sent_score'
Exception in get_mean_like_count: 'DataFrame' object has no attribute 'like_count'
Exception in get_mean_reply_count: 'DataFrame' object has no attribute 'reply_count'
Exception in get_mean_repost_count: 'DataFrame' object has no attribute 'repost_count'


100%|██████████| 1000/1000 [00:17<00:00, 58.52it/s]


ValueError: Columns must be same length as key

In [101]:
df_user_langs

,user_id,langs,nr_posts,date_min,date_max,mean_senti_score,mean_like_count,mean_reply_count,mean_repost_count,ERROR,"(nr_posts, date_min, date_max, mean_senti_score, mean_like_count, mean_reply_count, mean_repost_count, ERROR)"
13004,1129031,eng,40.0,1976-05-30 12:43:11.911,1976-05-31 15:06:01.437,0.730205,175.65,1.25,183.6,False,"[[eng], 40, 1976-05-30 12:43:11.911000, 1976-0..."
151527,90329,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 3, 1976-05-30 13:13:41.035000, 1976-05..."
434716,1473275,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[jpn], 30, 1976-05-31 14:48:01.025000, 1976-0..."
377522,2300499,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 5, 1976-05-30 12:40:51.023000, 1976-05..."
415626,2739245,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 1, 1976-05-31 14:51:21.355000, 1976-05..."
...,...,...,...,...,...,...,...,...,...,...,...
555470,1861174,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[deu], 3, 1976-05-30 13:48:31.059000, 1976-05..."
704339,1650182,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 1, 1976-05-30 13:00:12.003000, 1976-05..."
127404,2133501,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 1, 1976-05-30 12:58:21.212000, 1976-05..."
584672,2898268,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[jpn], 3, 1976-05-31 14:47:51.123000, 1976-05..."


In [104]:
first_index = df_user_langs.index[0]
df_user_langs.loc[first_index, "langs"] = list(get_metadata(df_user_langs.loc[first_index, 'user_id']))[0]
df_user_langs.loc[first_index, "nr_posts"] = list(get_metadata(df_user_langs.loc[first_index, 'user_id']))[1]
df_user_langs

,user_id,langs,nr_posts,date_min,date_max,mean_senti_score,mean_like_count,mean_reply_count,mean_repost_count,ERROR,"(nr_posts, date_min, date_max, mean_senti_score, mean_like_count, mean_reply_count, mean_repost_count, ERROR)"
13004,1129031,eng,40.0,1976-05-30 12:43:11.911,1976-05-31 15:06:01.437,0.730205,175.65,1.25,183.6,False,"[[eng], 40, 1976-05-30 12:43:11.911000, 1976-0..."
151527,90329,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 3, 1976-05-30 13:13:41.035000, 1976-05..."
434716,1473275,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[jpn], 30, 1976-05-31 14:48:01.025000, 1976-0..."
377522,2300499,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 5, 1976-05-30 12:40:51.023000, 1976-05..."
415626,2739245,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 1, 1976-05-31 14:51:21.355000, 1976-05..."
...,...,...,...,...,...,...,...,...,...,...,...
555470,1861174,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[deu], 3, 1976-05-30 13:48:31.059000, 1976-05..."
704339,1650182,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 1, 1976-05-30 13:00:12.003000, 1976-05..."
127404,2133501,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[eng], 1, 1976-05-30 12:58:21.212000, 1976-05..."
584672,2898268,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,"[[jpn], 3, 1976-05-31 14:47:51.123000, 1976-05..."


In [107]:
first_index = df_user_langs.index[100]
df_user_langs.loc[first_index, ['langs', 'nr_posts', 'date_min', 'date_max', 'mean_senti_score', 'mean_like_count', 'mean_reply_count', 'mean_repost_count', 'ERROR']] = get_metadata(df_user_langs.loc[first_index, 'user_id'])

KeyError: 'user_id'

In [96]:
df_user_langs.iloc[first_index]

IndexError: single positional indexer is out-of-bounds

In [66]:
df_user_langs.loc[0].user_id

np.int64(288811)

In [56]:

for user in os.listdir(PATH_USER_POSTS)[:10]:
    df_posts = pd.read_json(PATH_USER_POSTS + user, lines=True)
    langs = np.array([np.array(lang) for lang in df_posts.langs]).flatten()
    print(langs)
    

['jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn'
 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn'
 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn' 'jpn']


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2616,) + inhomogeneous part.

In [ ]:
df_posts.loc[:, ['like_count', 'reply_count', 'repost_count']].sum(axis=1).sort_values(ascending=False).head(10)

35    24
17    21
33    14
46    10
43     9
15     9
2      9
12     8
41     8
24     7
dtype: int64

In [ ]:
df_posts.iloc[35].text

'We had Wingspan designer @@58154 on the show!'